# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
#!pip install chromadb langchain langchain_community langchain_openai

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [30]:
llm = OpenAI(temperature=0)

In [31]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [32]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [33]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [34]:
from langchain_community.document_loaders import WebBaseLoader

In [35]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [36]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [37]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [38]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [39]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [40]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: 
Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer.
Final Answer: Biden nominated Ketanji Brown Jackson to serve on the Supreme Court and praised her as one of the nation's top legal minds.

> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "Biden nominated Ketanji Brown Jackson to serve on the Supreme Court and praised her as one of the nation's top legal minds."}

In [41]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter, so it may have different features and capabilities compared to flake8.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff offers a larger rule set and the ability to automatically fix lint violations, while also being able to be used as a formatter. It also supports more recent versions of Python and does not require the installation of Rust.
Thought: This information is helpful, but I should also consider the specific needs and preferences of my project.
Action: State of Union QA System
Action Input: "What are the specific features and capabilities of ruff that make it a better choice than flake8?"
Observation:  I don't know.
Thought: I should also consider the community support and popularity of each linter.
Action: State of Union QA System
Action Input: "What is the community support and popularity of ruff compared to flake8?"
Observation:  I don't know.
Thought: I should also look int

{'input': 'Why use ruff over flake8?',
 'output': 'It ultimately depends on the specific needs and preferences of the project, but ruff may be a better choice for its larger rule set, automatic fixing capabilities, and support for more recent versions of Python. However, it is important to also consider the community support and potential drawbacks of using ruff.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [42]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [43]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [44]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: 
Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "\nBiden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [45]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff offers a larger rule set and the ability to automatically fix lint violations, while also being able to be used as a formatter. It also supports more recent versions of Python and does not require the installation of Rust.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff offers a larger rule set and the ability to automatically fix lint violations, while also being able to be used as a formatter. It also supports more recent versions of Python and does not require the installation of Rust.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [46]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [47]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [48]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should check both tools to see if they have any information on Jupyter Notebooks.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff uses the colored crate to automatically detect whether the output stream supports color.
Thought: This doesn't answer the question about Jupyter Notebooks.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation:  No, the president did not mention a specific tool in the State of the Union.
Thought: I now know the final answer.
Final Answer: The president did not mention a specific tool in the State of the Union for running over Jupyter Notebooks.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'The president did not mention a specific tool in the State of the Union for running over Jupyter Notebooks.'}